# Классы
## Домашнее задание

### Вопросы по лекциям

#### 1.

Напишите название функции, которая является конструктором класса.

**Ответ:** `__init__`

#### 2.

На что указывает переменная `self`?

**Ответ:** `self` - это ссылка на объект, для которого вызван метод (к которому привязан вызванный метод)

#### 3.
С помощью какой функции можно проверить, что некая строка является именем одного из атрибутов объекта?

**Ответ:** `hasattr()`

#### 4.
Когда вызывается метод `__del__`? (относительно события удаления объекта)

**Ответ:** это неопределено, однако есть гарантия, что  `__del__` НЕ будет вызван пока есть хотя бы одна ссылка на объект

#### 5.
Верно ли, что атрибут класса перекрывает атрибут объекта?

**Ответ:** нет, при обращении к атрибуту через `self`, приоритет будет отдан атрибуту объекта

#### 6.
Можно ли атрибуты базового класса вызывать в дочернем классе? Если да, то напишите, нет ли исклчений?

**Ответ:** да, если имя аттрибута начинается не с двойного подчёркивания `__`

#### 7.
Объясните своими словами для чего нужен метод `super`.

**Ответ:** для получения ссылки на себя как на объект базового класса. Позволяет обращаться к атрибутам базового класса, что особенно актуально в случае их перекратия аттрибутами дочернего класса.

### Практика

1. Напишите класс `Fraction` для работы с дробями. Пусть дробь в нашем классе предстает в виде `числитель/знаменатель`. Дробное число должно создаваться по запросу `Fraction(a, b)`, где `a` – это числитель, а `b` – знаменатель дроби. 
2. Добавьте возможность сложения (сложения через оператор сложения) для дроби. Предполагается, что операция сложения может проводиться как только между дробями, так и между дробью и целым числом. Результат операции должен быть представлен в виде дроби.
3. Добавьте возможность взятия разности (вычитания через оператор вычитания) для дробей. Предполагается, что операция вычитания может проводиться как только для двух дробей, так и для дроби и целого числа. Результат операции должен быть представлен в виде дроби.
4. Добавьте возможность умножения (умножения через оператор умножения) для дробей. Предполагается, что операция умножения может проводиться как только для двух дробей, так и для дроби и целого числа. Результат операции должен быть представлен в виде дроби.
5. Добавьте возможность приведения дроби к целому числу через стандартную функцию `int()`.
6. Добавьте возможность приведения дроби к числу с плавающей точкой через стандартную функцию `float()`.
7. Создайте дочерний класс `OperationsOnFraction` и добавьте туда собственные методы `getint` и `getfloat`, которые будут возвращать целую часть дроби и представление дроби в виде числа с плавающей точкой соответственно. 
 

In [86]:
from typing import List, Union
from math import sqrt

# There are a bunch of tests at the end of the file. You may take a look at
# it to be confident that everything works perfectly

# Note: maybe fraction's simplification is overengineering here, because
# it is not required in the task, but it definitely makes class better

class Fraction:
    def __init__(self, numerator: int, denominator: int):
        assert denominator > 0  # negative denominators are also not allowed
        self.numerator = numerator
        self.denominator = denominator
        self.__simplify()
        
    def __eq__(self, other: Union[int, Fraction]):
        if isinstance(other, Fraction):
            # Both fraction are expected to be simplified
            return self.numerator == other.numerator and \
                   self.denominator == other.denominator
        else:
            return self.numerator == other * self.denominator
        
    def __add__(self, other: Union[int, Fraction]) -> "Fraction":
        if isinstance(other, Fraction):
            numerator = self.numerator * other.denominator + \
                        other.numerator * self.denominator
            denominator = self.denominator * other.denominator
            return Fraction(numerator, denominator)
        else:
            return Fraction(self.numerator + other * self.denominator,
                            self.denominator)
        
    def __radd__(self, other: int) -> "Fraction":
        return self.__add__(other)
    
    def __sub__(self, other: Union[int, Fraction]) -> "Fraction":
        return self.__add__(-other)
        
    def __rsub__(self, other: Union[int, Fraction]) -> "Fraction":
        return -self.__add__(-other)
    
    def __neg__(self) -> "Fraction":
        return Fraction(-self.numerator, self.denominator)
    
    def __mul__(self, other: Union[int, Fraction]) -> "Fraction":
        if isinstance(other, Fraction):
            return Fraction(self.numerator * other.numerator,
                            self.denominator * other.denominator)
        else:
            return Fraction(self.numerator * other, self.denominator)
    
    def __rmul__(self, other: Union[int, Fraction]) -> "Fraction":
        return self.__mul__(other)
    
    def __int__(self) -> int:
        # Can't be replaced with just:
        # ```return self.numerator // self.denominator```
        # because -7 // 3 = -3 (not -2!)
        r = abs(self.numerator) // self.denominator
        return -r if self.numerator < 0 else r
    
    def __float__(self) -> float:
        return self.numerator / self.denominator
    
    def __str__(self) -> str:
        if self.denominator == 1:
            return str(self.numerator)
        else:
            return f"{self.numerator}/{self.denominator}"
    
    def __simplify(self):
        if self.numerator == 0:
            # edge case
            self.denominator = 1
            return
            
        def to_primes(number: int) -> List[int]:
            """Represent the specified `number` as a list of primes"""
            primes = []
            while number % 2 == 0:
                primes.append(2)
                number //= 2
            for i in range(3, int(sqrt(number)) + 1, 2):
                while number % i == 0:
                    primes.append(i)
                    number = number // i
            if number > 2:
                primes.append(number)
            return primes
                    
        for prime in to_primes(min(abs(self.numerator), self.denominator)):
            if self.numerator % prime == 0 and self.denominator % prime == 0:
                self.numerator //= prime
                self.denominator //= prime
            
            
class OperationsOnFraction(Fraction):
    
    def getint(self) -> int:
        return super().__int__()
    
    def getfloat(self) -> int:
        return super().__float__()
            
# Some tests:

# Equals:
assert Fraction(3, 5) != Fraction(4, 5)
assert Fraction(3, 5) != Fraction(3, 7)
assert Fraction(3, 5) != -Fraction(3, 5)
assert Fraction(3, 5) == Fraction(3, 5)
assert Fraction(14, 7) == 2
assert 0 == Fraction(0, 4354)

# Simplifying:
assert Fraction(25, 5) == 5
assert Fraction(36, 84) == Fraction(3, 7)
assert Fraction(-36, 84) == Fraction(-3, 7)

# Negotiation:
assert -Fraction(6, 2) == -3
assert Fraction(-1, 3) == -(Fraction(1, 3))
assert Fraction(0, 3) == -Fraction(0, 3)

# Printing:
assert "5" == str(Fraction(25, 5))
assert "7/13" == str(Fraction(7, 13))
assert "3/7" == str(Fraction(90, 210))
assert "-3/7" == str(Fraction(-3, 7))
assert "-3/7" == str(-Fraction(3, 7))
assert "-3" == str(Fraction(-6, 2))
assert "0" == str(Fraction(0, 243))
assert "0" == str(-Fraction(0, 654))
    
# Add:
a = Fraction(7, 19)
assert Fraction(64, 19) == a + 3
assert a + 5 == 5 + a

b = Fraction(4, 21)
assert Fraction(223, 399) == a + b
assert b + a == a + b

# Sub:
a = Fraction(18, 4)
assert Fraction(10, 4) == a - 2
assert a - 2 == -(2 - a)

a = Fraction(2, 13)
b = Fraction(4, 13)
assert Fraction(2, 13) == b - a
assert a - b == -(b - a)
assert 0 == b - a - a

# Mul:
a = Fraction(5, 9)
assert Fraction(5, 3) == a * 3
assert -Fraction(5, 3) == a * (-3)
assert a * 3 == 3 * a
assert 0 == a * 0

a = Fraction(2, 7)
b = Fraction(9, 13)
assert Fraction(18, 91) == a * b
assert -Fraction(18, 91) == a * (-b)
assert b * a == a * b
assert a * b == (-a) * (-b)

# int()
assert 0 == int(Fraction(2, 7))
assert 0 == int(Fraction(6, 7))
assert 1 == int(Fraction(7, 7))
assert 3 == int(Fraction(21, 7))
assert -3 == int(Fraction(-22, 7))

# float()
assert 7 / 19 == float(Fraction(7, 19))
assert -19 / 7 == float(Fraction(-19, 7))
assert 0 == float(Fraction(0, 7))

a = OperationsOnFraction(9, 4)
assert 2 == a.getint()
assert 9 / 4 == a.getfloat()

b = OperationsOnFraction(-17, 3)
assert -5 == b.getint()
assert -17 / 3 == b.getfloat()

print("All tests passed!")

All tests passed!
